In [ ]:
import PySimpleGUI as sg
from mtgsdk import Card
from PIL import Image
import requests
import io
import speech_recognition as sr

# デザインテーマの設定
sg.theme('DarkBlue')

# GUIレイアウトの設定
layout = [
    [sg.Text('カード名'), sg.InputText(key='card_name'), sg.Button('音声認識')],
    [sg.Button('検索'), sg.Button('終了')],
    [sg.Multiline(size=(40, 10), key='output')],
    [sg.Image(key='image')],
    [sg.Button('Next'),sg.Text(key='card_number')]
]

# GUIウィンドウの生成
window = sg.Window('マジック：ザ・ギャザリングのカード検索', layout)
# 検索結果配列
cards = []
#日本語カード用配列
japanesecards = []
#現在表示中のカード
current_card_index = 0

def get_crad_data():
    window['card_number'].update(f"{current_card_index+1}/{len(japanesecards)}")
    current_card = japanesecards[current_card_index]
        
    #カード説明を更新
    window['output'].update(f"カード名: {current_card['name']}\n"
                                f"カードタイプ: {current_card['type']}\n"
                                f"カードテキスト: {current_card['text']}\n")
    # 画像を取得
    if current_card.get('imageUrl'):
        print(current_card['imageUrl'])
        response = requests.get(current_card['imageUrl'],verify=False)
        #画像を更新
    if response.status_code == 200:
        #PNGに変換
        img = Image.open(io.BytesIO(response.content))
        png_bytes = io.BytesIO()
        img.save(png_bytes, format="PNG")
        png_bytes.seek(0)
        #画像を出力
        window['image'].update(data=png_bytes.read())
            
# イベントループ
while True:
    event, values = window.read()
    if event in (None, '終了'):
        break
    
    if event in '検索':
        #配列の初期化
        cards = []
        japanesecards = []
        # カード検索
        if  not cards:
            cards = Card.where(name=values['card_name']).where(language='japanese').all()
            # 現在表示しているカード
            current_card_index = 0
        #日本語のデータのみ抽出
        if  not japanesecards:
            for card in cards:
                if card.foreign_names:
                    for foreign_name in card.foreign_names:
                        if foreign_name['language'] == 'Japanese':
                            if foreign_name.get('name') and foreign_name.get('type') and foreign_name.get('text') and foreign_name.get('imageUrl'):
                                japanesecard = {
                                    'name': foreign_name['name'],
                                    'type': foreign_name['type'],
                                    'text': foreign_name['text'],
                                    'imageUrl': foreign_name['imageUrl']
                                }
                                #データを配列にappend
                                japanesecards.append(japanesecard)
        get_crad_data()
        
    if event in '音声認識':
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            print('音声入力中...')
            audio = recognizer.listen(source)
        try:
            card_name = recognizer.recognize_google(audio, language='ja-JP')
            window['card_name'].update(card_name)
        except sr.UnknownValueError:
            print('音声を認識できませんでした')
        except sr.RequestError as e:
            print(f'リクエストに失敗しました: {e}')
            
    #次のカードに変更
    if event in 'Next':
        current_card_index += 1
        if current_card_index >= len(japanesecards):
            current_card_index = 0
        current_card = japanesecards[current_card_index]
        get_crad_data()

window.close()